In [1]:
import numpy as np
from mindquantum.core.gates import RX, RY, RZ, H, X, Y, Z, CNOT
from mindquantum.core.circuit import Circuit
import mindspore as ms
from mindquantum.simulator import  Simulator
from mindquantum.core.gates import GroupedPauli
from mindquantum.core.operators import TimeEvolution,QubitOperator
from mindquantum.core.parameterresolver import PRConvertible,PRGenerator,ParameterResolver
from DQAS_tool import generate_pauli_string,one_hot,unbound_opeartor_pool
from mindquantum.core.gates import RotPauliString
from mindquantum.core.gates import UnivMathGate
from mindspore import Tensor, ops
from mindquantum.core.circuit import UN
from mindquantum.core.operators import Hamiltonian             # 导入Hamiltonian模块，用于构建哈密顿量
from mindquantum.framework import MQLayer
from mindspore.nn import  TrainOneStepCell
from mindspore.nn import SoftmaxCrossEntropyWithLogits                         # 导入SoftmaxCrossEntropyWithLogits模块，用于定义损失函数
from mindspore.nn import Adam                                                  # 导入Adam模块用于定义优化参数
from mindspore.train import Accuracy, Model, LossMonitor                       # 导入Accuracy模块，用于评估预测准确率
import mindspore as ms
from mindspore import Parameter, Tensor
from mindspore.dataset import NumpySlicesDataset
from torch.utils.data import DataLoader# 导入NumpySlicesDataset模块，用于创建模型可以识别的数据集
import sys
sys.path.append('..')
from data_processing import X_train,X_test,y_train,y_test
from mindquantum.algorithm.nisq import HardwareEfficientAnsatz     
num_layer = 3
# 定义标准差和形状
stddev = 0.02
shape_parametized = 8
shape_unparametized = 4
shape_nnp = (num_layer, shape_parametized)
shape_stp = (num_layer, shape_parametized+shape_unparametized)

shape_stp = (num_layer, shape_parametized)

rtype = np.float64
ctype = np.complex128
# 使用 numpy 生成随机数矩阵
np.random.seed(10)
nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp).astype(rtype)
stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp).astype(rtype)
# #Operator Pool
unbound_opeartor_pool = [generate_pauli_string(n=8,seed=i)[0] for i in range(shape_parametized)]
bound_opeartor_pool = [generate_pauli_string(n=8,seed=i)[1] for i in range(shape_parametized,shape_parametized+shape_unparametized)]

In [11]:
from DQAS_tool import Mindspore_ansatz,loss_fn,vag_nnp,sampling_from_structure,vag_nnp,vag_nnp_function,sampling_from_structure
from mindquantum.framework import MQOps
import mindspore.nn as nn

In [114]:
import mindspore.numpy as mnp
def nmf_gradient(structures:np.array, oh:ms.Tensor,num_layer: int,size_pool:int):
    """
    使用 MindSpore 实现蒙特卡洛梯度计算。
    """
      # Step 1: 获取选择的索引
    choice = ops.Argmax(axis=-1)(oh)
    # Step 2: 计算概率
    softmax = ops.Softmax(axis=-1)
    prob = softmax(ms.Tensor(structures))
    # Step 3: 获取概率矩阵中的值
    indices = mnp.stack((mnp.arange(num_layer, dtype=ms.int64), choice), axis=1)
    prob = ops.GatherNd()(prob, indices)
    # Step 4: 变换概率矩阵
    prob = prob.reshape(-1, 1)
    prob = ops.Tile()(prob, (1, size_pool))
    
    # Step 5: 生成蒙特卡洛梯度
    gradient = ops.TensorScatterAdd()(Tensor(-prob, ms.float64), indices, mnp.ones((num_layer,), dtype=ms.float64))
    return gradient
    
    
# 对向量化版本的封装
# nmf_gradient_vmap = ops.vmap(nmf_gradient, in_axes=(None, 0, None, None))

def best_from_structure(structures: np.array):
    return ops.Argmax(axis=-1)(ms.Tensor(structures))

In [ ]:
import numpy as np
import tensorcircuit as tc
import tensorflow as tf
K = tc.set_backend("tensorflow")
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.06, 100, 0.5)
structure_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(0.12))
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(lr))
verbose = True
# 设置超参数
epochs = 100
batch_size=50
nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp).astype(rtype)
stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp).astype(rtype)
# 定义学习率衰减
# lr = nn.ExponentialDecayLR(learning_rate=0.06, decay_rate=0.5, decay_steps=100)
# # 定义优化器
# structure_params = Parameter(Tensor(stp), name='Structure_params')
# ansatz_params    = Parameter(Tensor(nnp), name='Ansatz_params')

# structure_opt = nn.Adam(params=[structure_params], learning_rate=0.12)
# network_opt = nn.Adam(params=[ansatz_params], learning_rate=lr)
avcost1 = 0

ops_onehot = ops.OneHot(axis=-1)

for epoch in range(epochs):  # 更新结构参数的迭代
    avcost2 = avcost1
    costl = []
    tmp = np.stack([sampling_from_structure(stp,3,8) for _ in range(batch_size)])
    batch_structure = ops_onehot(ms.Tensor(tmp),8,ms.Tensor(1),ms.Tensor(0))
    # print(tmp,batch_structure)
    loss_value = []
    grad_nnps = []
    grad_stps = []
    
    for i in batch_structure:          
        infd, grad_nnp = vag_nnp(Structure_params=i,Ansatz_params=nnp, n_layer=3,n_qbits=8)(ms.Tensor(X_train),ms.Tensor(y_train))
        gs = nmf_gradient(structures=stp,oh=i,num_layer=3,size_pool=8)
        
        #print(infd,grad_nnp)
        loss_value.append(infd)
        grad_nnps.append(grad_nnp[0])
        grad_stps.append(gs)
    
    infd = ops.stack(loss_value)
    gnnp = ops.addn(grad_nnps)
    gstp = [(infd[i] - avcost2) * grad_stps[i] for i in range(infd.shape[0])]
    gstp_averge = ops.addn(gstp) / infd.shape[0]
    avcost1 = sum(infd) / infd.shape[0]
    
    gnnp_tf = tf.convert_to_tensor(gnnp.reshape(nnp.shape).asnumpy(),dtype=tf.float64)
    nnp_tf = tf.convert_to_tensor(nnp,dtype=tf.float64)
    gstp_averge_tf = tf.convert_to_tensor(gstp_averge.reshape(stp.shape).asnumpy(),dtype=tf.float64)
    stp_tf = tf.convert_to_tensor(stp,dtype=tf.float64)
    # 更新参数
    nnp_tf = network_opt.update(gnnp_tf, nnp_tf)
    stp_tf = structure_opt.update(gstp_averge_tf, stp_tf) 
    
    nnp = nnp_tf.numpy()
    stp = stp_tf.numpy()
    
    
    if epoch % 5 == 0 or epoch == epochs - 1:
        print("----------epoch %s-----------" % epoch)
        print(
            "batched average loss: ",
            avcost1,
        )
    
        if verbose:
            print(
                "strcuture parameter: \n",
                stp,
                "\n network parameter: \n",
                nnp,
            )
        
        cand_preset = best_from_structure(stp)
        print("best candidates so far:",cand_preset)
        # print(
        #     "corresponding weights for each gate:",
        #     [K.numpy(nnp[j, i]) if i < 6 else 0.0 for j, i in enumerate(cand_preset)],
        # )
    
    
    
    
    
    
    
    
    
    


In [118]:
unbound_opeartor_pool

['I0 I1 X2 Z3 I4 I5 Z6 I7',
 'Y0 X1 Z2 X3 I4 I5 I6 I7',
 'X0 X1 X2 Z3 Y4 Z5 Z6 Y7',
 'Y0 Y1 Z2 I3 X4 X5 I6 Z7',
 'Y0 Z1 X2 I3 I4 Y5 X6 X7',
 'Z0 Z1 X2 I3 Y4 X5 Y6 X7',
 'X0 I1 Z2 X3 X4 Y5 I6 Z7',
 'Z0 Y1 I2 X3 X4 X5 Z6 X7']

In [107]:
gnnp.reshape(nnp.shape).asnumpy()
gnnp_tf = tf.convert_to_tensor(gnnp.reshape(nnp.shape).asnumpy(),dtype=tf.float64)
nnp_tf = tf.convert_to_tensor(nnp,dtype=tf.float64)


In [113]:
gnnp_tf.numpy()

array([[ 5.72586032e-08,  1.36091495e-02,  7.08998181e-03,
         1.16583984e-03,  5.36890607e-03, -1.50375925e-07,
        -1.36224274e-03, -2.24929256e-03],
       [ 7.57133023e-08,  1.36107672e-02,  1.37904137e-02,
         3.07811541e-04,  2.60351575e-03, -2.02096544e-08,
        -2.31005158e-03, -1.54890760e-03],
       [ 3.04580894e-09,  1.84652936e-02,  7.09007960e-03,
        -5.39380533e-04,  2.60332157e-03, -2.56724313e-08,
        -1.60007132e-03, -1.54892216e-03]])

In [109]:
nnp = network_opt.update(gnnp_tf, nnp_tf)

In [110]:
nnp

<tf.Tensor: shape=(3, 8), dtype=float64, numpy=
array([[-9.55692689e-03, -5.76082439e-02, -8.63090953e-02,
        -7.81198747e-02, -9.76621702e-02, -4.05456351e-03,
         7.12617415e-02,  7.33694538e-02],
       [ 1.55234675e-02, -5.57504269e-02, -1.00629721e-01,
        -7.62470566e-02, -5.06417149e-02,  7.19465431e-06,
         8.59827429e-02,  2.29825579e-02],
       [ 3.40375434e-02, -6.60809879e-02, -7.67932112e-02,
         8.68686691e-02, -7.68064266e-02,  3.62715725e-03,
         5.30593249e-02,  8.73690202e-02]])>

In [86]:

grads = gnnp.reshape(nnp.shape)
grads = ms.Tensor(grads,ms.float32)

structure_opt = nn.Adam(params=[Tensor(stp,ms.float32)], learning_rate=0.12)
structure_params(grads)

TypeError: For 'Optimizer', all elements of the argument 'parameters' must be 'Parameter' or 'dict', please check the 'parameters'.

In [87]:
import mindspore as ms
import mindspore.nn as nn
from mindspore import Tensor, Parameter

# 假设 stp 和 nnp 已经定义
stp = np.random.randn(8)  # 示例数据
nnp = np.random.randn(8)  # 示例数据

# 假设 gnnp 已经计算出来
gnnp = Tensor(np.random.randn(8), ms.float32)  # 示例数据

# 将 gnnp 重新调整形状并转换为 Tensor
grads = gnnp.reshape(nnp.shape)
grads = Tensor(grads, ms.float32)

# 将 stp 转换为 Parameter
structure_param = Parameter(Tensor(stp, ms.float32), name="structure_param")

# 定义优化器
structure_opt = nn.Adam(params=[structure_param], learning_rate=0.12)

# 使用优化器更新参数


# 打印更新后的参数
print("Updated structure_param:", structure_param.asnumpy())

RuntimeError: The types of arguments in Map must be consistent, but the types of arguments are inconsistent.
There are 7 inputs of `map`, corresponding type info:
In file /opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindspore/nn/optim/adam.py:901
                            success = self.map_(F.partial(_adam_opt, self.opt, self.sparse_opt, self._ps_push,
                                      ^
.
The type of the second argument in Map is: List[Tensor[Float32]].
The type of the third argument in Map is: Tuple[Ref[Tensor[Float32]]].
The type of the 4th argument in Map is: Tuple[Ref[Tensor[Float32]]].
The type of the 5th argument in Map is: Tuple[Ref[Tensor[Float32]]].
The type of the 6th argument in Map is: Tuple[Bool].
The type of the 7th argument in Map is: Tuple[Bool].

----------------------------------------------------
- C++ Call Stack: (For framework developers)
----------------------------------------------------
mindspore/ccsrc/frontend/operator/composite/map.cc:26

In [75]:
gnnp.reshape(nnp.shape)

Tensor(shape=[3, 8], dtype=Float32, value=
[[-9.88132456e-08,  1.32359611e-02, -1.76628027e-03 ...  5.35264235e-07, -4.51928005e-03, -2.10772082e-03],
 [-2.20070717e-07,  1.32363131e-02, -1.25867198e-03 ... -1.39021199e-06, -3.57847963e-03, -2.90810829e-03],
 [-1.14492039e-07,  1.78225935e-02, -1.25893648e-03 ...  3.06085539e-08, -7.34172668e-03, -2.10750639e-03]])

In [74]:
nnp

array([[-0.01122259,  0.01832192,  0.0024974 , -0.00064747, -0.05200137,
         0.01730596,  0.01005961,  0.00038118],
       [-0.02940138, -0.01214009, -0.01184792,  0.0424261 , -0.04052099,
        -0.04069649, -0.0063086 ,  0.00610193],
       [ 0.00237795, -0.00130611,  0.02833612,  0.0174096 , -0.02848663,
         0.04429366,  0.053757  ,  0.02116067]])

In [70]:
gstp_averge

Tensor(shape=[3, 8], dtype=Float64, value=
[[ 7.02791789e-02, -1.11342516e-01,  7.02871897e-02 ...  2.51894877e-01,  7.02913501e-02, -1.11342516e-01],
 [-1.13839345e-01, -1.13839345e-01, -1.13839345e-01 ...  4.31045921e-01, -1.13839345e-01,  6.77940914e-02],
 [ 6.72984868e-02,  6.72980577e-02, -1.14335379e-01 ... -1.14335379e-01,  2.48890272e-01, -1.14335379e-01]])

In [63]:
sum(infd)

Tensor(shape=[], dtype=Float32, value= 4.54124)

In [61]:
ops.addn(gstp)/7

Tensor(shape=[3, 8], dtype=Float64, value=
[[-8.05453571e-02,  1.78959055e-01,  4.92015483e-02 ... -8.05453571e-02, -8.05453571e-02,  4.92067510e-02],
 [-8.02022707e-02, -8.02022707e-02,  4.95423953e-02 ... -8.02022707e-02,  3.09048212e-01, -8.02022707e-02],
 [-8.20206445e-02, -8.20206445e-02,  4.77262609e-02 ... -8.20206445e-02,  1.77476964e-01,  4.77314635e-02]])

In [47]:
infd

Tensor(shape=[5], dtype=Float32, value= [ 9.08508897e-01,  9.08438444e-01,  9.08429801e-01,  9.08451319e-01,  9.08427000e-01])

In [ ]:
ops.addn(grad_nnps)

In [42]:
ops_onehot = ops.OneHot(axis=-1)
batch_size=10
tmp = np.stack([sampling_from_structure(stp,3,8) for _ in range(batch_size)])
batch_structure = ops_onehot(ms.Tensor(tmp),8,ms.Tensor(1),ms.Tensor(0))



In [50]:
choice = ops.Argmax(axis=-1)(batch_structure)

In [51]:
nmf_gradient(structures=stp,oh=batch_structure[0],num_layer=3,size_pool=8)

Tensor(shape=[3, 8], dtype=Float64, value=
[[-1.26967952e-01,  8.73032048e-01, -1.26967952e-01 ... -1.26967952e-01, -1.26967952e-01, -1.26967952e-01],
 [-1.26017213e-01, -1.26017213e-01, -1.26017213e-01 ... -1.26017213e-01, -1.26017213e-01, -1.26017213e-01],
 [-1.28127679e-01, -1.28127679e-01,  8.71872321e-01 ... -1.28127679e-01, -1.28127679e-01, -1.28127679e-01]])

In [12]:
#ansatz =Mindspore_ansatz(Structure_p=stp,n_layer=2,n_qbits=8)

ansatz = HardwareEfficientAnsatz(8, single_rot_gate_seq=[RY], entangle_gate=X, depth=2).circuit   
ansatz = encoder + ansatz.as_ansatz()
sim = Simulator(backend='mqvector', n_qubits=8)
hams = [Hamiltonian(QubitOperator(f'Z{i}')) for i in [0, 1]]
grad_ops = sim.get_expectation_with_grad(hams, ansatz)
Myops = MQOps(grad_ops)


def forward_fn(encode_p, ansatz_p, y_label):
    eval_obserables = Myops(encode_p, ansatz_p)
    loss = loss_fn(eval_obserables, y_label)
    return loss
grad_fn = ms.value_and_grad(fn=forward_fn,grad_position=None,weights=nnp)


In [ ]:
index=140
grad_fn(ms.Tensor(X_train[0:index]),nnp, ms.Tensor(y_train[0:index]))

In [ ]:
ansatz

In [ ]:
import numpy as np

a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
c = np.array([7, 8, 9])

# 沿着新轴堆叠数组
result = np.stack((a, b, c), axis=0)
print(result)

In [41]:
def forward_fn(Net,loss_fun,x, y):
    z = Net(x)
    #print(z)
    loss = loss_fun(z, y)
    return loss

def my_fun(Structure_params:np.array,Ansatz_params:np.array):
    ansatz = Mindspore_ansatz(Structure_params,3,8)
    sim = Simulator(backend='mqvector',n_qubits=8)
    hams = [Hamiltonian(QubitOperator(f'Z{i}')) for i in [0,1]]
    grad_ops= sim.get_expectation_with_grad(hams,ansatz)
    loss = ms.nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean') # 定义损失函数  
    # f,_,grad_ansatz_params = grad_ops(X_train[0],Ansatz_params) #grad_ansatz_params
    QuantumNet = MQLayer(grad_ops)
    opti = Adam(QuantumNet.trainable_params(), learning_rate=0.5)     # 需要优化的是Quantumnet中可训练的参数，学习率设为0.5
    grad_fn = ms.value_and_grad(forward_fn, None, weights=QuantumNet.trainable_params())
    
    value,grad_ansatz_params = grad_fn(ms.Tensor(X_train), ms.Tensor(y_train))
    
    
    
    

In [ ]:
net = Mindspore_ansatz(Structure_p=stp,Ansatz_p=nnp,n_layer=3,n_qbits=8)

In [ ]:
print(net.trainable_params())


In [20]:
loss_fn = ms.nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean') # 定义损失函数    
def forward_fn(x, y):
    z = net(x)
    print(z)
    loss = loss_fn(z, y)
    return loss

In [ ]:
grad_fn = ms.value_and_grad(forward_fn, None, weights=net.trainable_params())
loss, grads = grad_fn(X_train[0],y_train[0])
print(grads)
